# Automatic check whether osm fountains with wikidata ref have coordinates in Wikidata
The following script takes the Q-# obtained from OSM and checks whether those fountains have coordinates in WikiData.
If not, the osm coordinates will be added to wikidata with reference to the osm license
https://github.com/water-fountains/import2wikidata/issues/15.

## Initialize environment

In [1]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import io
import numpy as np
from urllib.request import urlopen
import json
from math import *
from platform import python_version
print("Python v "+python_version())
#https://github.com/paulhoule/gastrodon/issues/7 
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
from matplotlib import pyplot


191124_195208
Python v 3.6.5


In [3]:
#@prefix wikibase: <wikibase: <http://wikiba.se/ontology#> .
prefixes=inline("""
   @prefix wd: <http://www.wikidata.org/entity/> .
   @prefix wdt: <http://www.wikidata.org/prop/direct/> .
   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
   @prefix p: <http://www.wikidata.org/prop/> .
   @prefix psv: <http://www.wikidata.org/prop/statement/value/> .
   @prefix wikibase: <http://wikiba.se/ontology#> .
""").graph
endpoint=RemoteEndpoint(
   #"https://query.wikidata.org/sparql"
    "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
   ,prefixes=prefixes
)

## Load data

In [36]:
df = pd.read_csv("osmFountainsWithWikidataReference191124_092526.csv")

In [37]:
df.head()

,lat,lon,wikidata
0,47.402830,8.499940,Q27230124
1,47.409292,8.494851,Q27229864
2,47.338411,8.532262,Q27230215
3,47.338468,8.530644,Q27229839
4,47.344198,8.529788,Q27229951


In [38]:
len(df)

267

## Identify already existing fountains
### Query fountains from Wikidata

In [39]:
def coordsFunction(row):    
    itemid = row['wikidata']    
    query = ("""SELECT ?lat ?lon
    WHERE
    {{
      wd:%s p:P625 ?statement.
    ?statement psv:P625 ?node.
    ?node wikibase:geoLatitude ?lat.
    ?node wikibase:geoLongitude ?lon.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }}""")%itemid    
    print(itemid)
    data = endpoint.select(query)
    #print(itemid+" "+data['lat'])
    wdDf = pd.DataFrame(data)
    #wdDf
    if  len(wdDf.values) == 0:
        return 'noCoord'
    return wdDf.values[0][0]


In [40]:
# Perform query on each dw
result = df.apply(coordsFunction,axis=1)

Q27230124
Q27229864
Q27230215
Q27229839
Q27229951
Q27229857
Q27230182
Q27229899
Q55169625
Q27229818
Q27230098
Q27229885
Q27230198
Q27230202
Q55166802
Q27229775
Q27230014
Q27230180
Q27229957
Q55166749
Q27230049
Q27229889
Q27229977
Q27229956
Q27229909
Q27229959
Q27229964
Q27230021
Q27229888
Q55165905
Q55169659
Q27229914
Q27230113
Q27229971
Q55166163
Q27229965
Q27230185
Q27229658
Q55166819
Q27229783
Q27229890
Q27229960
Q27229877
Q27229787
Q55166799
Q27229689
Q27229892
Q27230209
Q27230353
Q55170092
Q27229904
Q55169693
Q55170009
Q55170006
Q55170003
Q55170000
Q55169998
Q55169995
Q55170013
Q55170762
Q27230047
Q27229670
Q27230203
Q27229779
Q27230229
Q27229659
Q27230090
Q27230201
Q27230102
Q27229685
Q27230181
Q27230071
Q27230018
Q27229881
Q55166813
Q27230011
Q27229958
Q27230126
Q27230134
Q27229662
Q27229669
Q27229660
Q27229664
Q27229945
Q27230070
Q27229827
Q55166825
Q27230216
Q64015724
Q55166416
Q27229887
Q27230178
Q27229891
Q27229785
Q55166615
Q55166804
Q27229858
Q27230094
Q27230183
Q27230037


In [41]:
result.head()

0    47.4028
1    47.4093
2    47.3384
3    47.3385
4    47.3442
dtype: object

In [42]:
query_result = pd.concat([df,result],ignore_index=True, axis=1)

In [43]:
#print(query_string)
print("\n\nTotal number of rows incl. duplicates "+str(len(query_result))+" size "+str(query_result.size))



Total number of rows incl. duplicates 267 size 1068


In [44]:
query_result.head()

,0,1,2,3
0,47.402830,8.499940,Q27230124,47.4028
1,47.409292,8.494851,Q27229864,47.4093
2,47.338411,8.532262,Q27230215,47.3384
3,47.338468,8.530644,Q27229839,47.3385
4,47.344198,8.529788,Q27229951,47.3442


## Create Quickstatement commands from data
### Helper functions to format content according to Quickstatements v1 syntax

In [47]:
def process_coordinates(x, y):
    # format geographic coordinates
    return '@{1:1.8f}/{0:1.8f}'.format(x,y)


def createline(lines, item, prop, value, extra, qualifiers=[]):
    # general function to create Quickstatement v1 commands
    if value != '' and value != '""':
        statement = '{}\t{}\t{}'.format(item, prop, value)
        if len(qualifiers):
            # append qualifiers if applicable
            for q in qualifiers:
                statement += '\t{}\t{}'.format(q['prop'], q['value'])
        statement += extra
        statement += '\n'
        lines.append(statement)
    return lines

### Create statements, taking care not to overwrite existing data

In [59]:
statedId = "\tS248\tQ1224853"
# initialize command storage list
lines = []
i=0
j=0
for index, row in query_result.iterrows():
    i+=1

    if row[3] == 'noCoord':
        j+=1
        # coordinates
        print(str(j)+":\torphan coordinates will be added for https://www.wikidata.org/wiki/"+row[2]+" in pos "+str(i))
        #TODO how to add ' target="_blank" '
        #TODO how to make only the Q-# visible, but the whole link anyway
        lines = createline(lines, row[2], 'P625', process_coordinates(row[1], row[0]),statedId)
    #else:
    #    print(str(i)+": "+" ignore "+row[2])
        

1:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q64015724 in pos 89
2:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q63715939 in pos 101
3:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q64174346 in pos 250
4:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q64174640 in pos 251
5:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q65548962 in pos 261
6:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q74369417 in pos 262
7:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q67588104 in pos 264
8:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q67588045 in pos 265
9:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q67587947 in pos 266
10:	orphan coordinates will be added for https://www.wikidata.org/wiki/Q74171778 in pos 267


# Write commands to file

In [63]:
quickStatFileName = "quickstatement_commands_locOrphan_fountain_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatFileName, "w", encoding='utf8') as f:
    f.writelines(lines)
print("wrote '"+quickStatFileName+"' with "+str(len(lines))+" lines")

wrote 'quickstatement_commands_locOrphan_fountain_191125_072628.txt' with 10 lines


# Import into Wikidata
- Go to https://tools.wmflabs.org/wikidata-todo/quick_statements.php.
- Authenticate yourself with your Wikidata account.
- Copy and paste the contents of quickstatement_commands*.txt into the blank field, and run the commands

Caveats
- do not interrupt this - it may take some time !  
- if you restart it, you may produce duplicates - if you think you have to restart, first wait half an hour !

see ../20191030_1600_import.png

...
58. Processing Q72935495 (Q72935495 Lde "Brunnen (Seelöwe-Planschbecken )")
59. Processing Q72935495 (Q72935495 P137 Q27229237)

All done!.

In [15]:
# it may well take half an hour until it works and becomes visible on https://query.wikidata.org/

In [61]:
#TODO Take the opposite approach: instead of querying osm first, do sparql to get the fountain instances without coordinates